##### Copyright 2023 The MediaPipe Authors. All Rights Reserved.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Pose Landmarks Detection with MediaPipe Tasks

This notebook shows you how to use MediaPipe Tasks Python API to detect pose landmarks from images.

## Preparation

Let's start with installing MediaPipe.


In [7]:
!pip install -q mediapipe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 44.4 MB/s eta 0:00:00


Then download an off-the-shelf model bundle. Check out the [MediaPipe documentation](https://developers.google.com/mediapipe/solutions/vision/pose_landmarker#models) for more information about this model bundle.

In [8]:
!wget -O pose_landmarker.task -q https://storage.googleapis.com/mediapipe-models/pose_landmarker/pose_landmarker_heavy/float16/1/pose_landmarker_heavy.task

## Visualization utilities

In [ ]:
#@markdown To better demonstrate the Pose Landmarker API, we have created a set of visualization tools that will be used in this colab. These will draw the landmarks on a detect person, as well as the expected connections between those markers.

from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np


def draw_landmarks_on_image(rgb_image, detection_result):
  pose_landmarks_list = detection_result.pose_landmarks
  annotated_image = np.copy(rgb_image)

  # Loop through the detected poses to visualize.
  for idx in range(len(pose_landmarks_list)):
    pose_landmarks = pose_landmarks_list[idx]

    # Draw the pose landmarks.
    pose_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    pose_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in pose_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      pose_landmarks_proto,
      solutions.pose.POSE_CONNECTIONS,
      solutions.drawing_styles.get_default_pose_landmarks_style())
  return annotated_image

## Download test image

To demonstrate the Pose Landmarker API, you can download a sample image using the follow code. The image is from [Pixabay](https://pixabay.com/photos/girl-woman-fitness-beautiful-smile-4051811/).

In [ ]:
!wget -q -O image.jpg https://cdn.pixabay.com/photo/2019/03/12/20/39/girl-4051811_960_720.jpg

import cv2
from google.colab.patches import cv2_imshow

img = cv2.imread("image.jpg")
cv2_imshow(img)

Optionally, you can upload your own image. If you want to do so, uncomment and run the cell below.

In [ ]:
# from google.colab import files
# uploaded = files.upload()

# for filename in uploaded:
#   content = uploaded[filename]
#   with open(filename, 'wb') as f:
#     f.write(content)

# if len(uploaded.keys()):
#   IMAGE_FILE = next(iter(uploaded))
#   print('Uploaded file:', IMAGE_FILE)

## Running inference and visualizing the results

The final step is to run pose landmark detection on your selected image. This involves creating your PoseLandmarker object, loading your image, running detection, and finally, the optional step of displaying the image with visualizations.

Check out the [MediaPipe documentation](https://developers.google.com/mediapipe/solutions/vision/pose_landmarker/python) to learn more about configuration options that this solution supports.


In [ ]:
# STEP 1: Import the necessary modules.
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# STEP 2: Create an PoseLandmarker object.
base_options = python.BaseOptions(model_asset_path='pose_landmarker.task')
options = vision.PoseLandmarkerOptions(
    base_options=base_options,
    output_segmentation_masks=True)
detector = vision.PoseLandmarker.create_from_options(options)

# STEP 3: Load the input image.
image = mp.Image.create_from_file("image.jpg")

# STEP 4: Detect pose landmarks from the input image.
detection_result = detector.detect(image)

# STEP 5: Process the detection result. In this case, visualize it.
annotated_image = draw_landmarks_on_image(image.numpy_view(), detection_result)
cv2_imshow(cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR))

Visualize the pose segmentation mask.

In [ ]:
segmentation_mask = detection_result.segmentation_masks[0].numpy_view()
visualized_mask = np.repeat(segmentation_mask[:, :, np.newaxis], 3, axis=2) * 255
cv2_imshow(visualized_mask)

In [2]:
import cv2

In [ ]:
KEY_LANDMARKS = {
    "LEFT_HIP": 23, "RIGHT_HIP": 24,
    "LEFT_KNEE": 25, "RIGHT_KNEE": 26,
    "LEFT_ANKLE": 27, "RIGHT_ANKLE": 28,
    "LEFT_SHOULDER": 11, "RIGHT_SHOULDER": 12
}

In [3]:
  # Lecture de la vidéo avec OpenCV
  cap = cv2.VideoCapture("IMG_0753.MOV")


In [ ]:
def new_extract_squat_data(results, prev_landmark):
  data = {"landmarks": {}, "angles": {}}

  if results.pose_landmarks:
    landmarks = results.pose_landmarks.landmark

    # Récupération des coordonnées des points clés
    for name, idx in KEY_LANDMARKS.items():
        data["landmarks"][name] = {
            "x": landmarks[idx].x,
            "y": landmarks[idx].y,
            "z": landmarks[idx].z
        }
    # Calcul des angles et Incrémenter que s'il y a une différence entre la position actuelle et celle d'avant
    if has_movement(data["landmarks"], prev_landmark):
      # Calcul des angles importants pour le squat
      data["angles"]["left_knee"] = calculate_angle(
          [landmarks[KEY_LANDMARKS["LEFT_HIP"]].x, landmarks[KEY_LANDMARKS["LEFT_HIP"]].y],
          [landmarks[KEY_LANDMARKS["LEFT_KNEE"]].x, landmarks[KEY_LANDMARKS["LEFT_KNEE"]].y],
          [landmarks[KEY_LANDMARKS["LEFT_ANKLE"]].x, landmarks[KEY_LANDMARKS["LEFT_ANKLE"]].y]
      )

      data["angles"]["right_knee"] = calculate_angle(
          [landmarks[KEY_LANDMARKS["RIGHT_HIP"]].x, landmarks[KEY_LANDMARKS["RIGHT_HIP"]].y],
          [landmarks[KEY_LANDMARKS["RIGHT_KNEE"]].x, landmarks[KEY_LANDMARKS["RIGHT_KNEE"]].y],
          [landmarks[KEY_LANDMARKS["RIGHT_ANKLE"]].x, landmarks[KEY_LANDMARKS["RIGHT_ANKLE"]].y]
      )

      data["angles"]["back_angle"] = calculate_angle(
          [landmarks[KEY_LANDMARKS["LEFT_SHOULDER"]].x, landmarks[KEY_LANDMARKS["LEFT_SHOULDER"]].y],
          [landmarks[KEY_LANDMARKS["LEFT_HIP"]].x, landmarks[KEY_LANDMARKS["LEFT_HIP"]].y],
          [landmarks[KEY_LANDMARKS["LEFT_KNEE"]].x, landmarks[KEY_LANDMARKS["LEFT_KNEE"]].y]
      )
      prev_landmark = data["landmarks"]
  return data["angles"], prev_landmark

In [ ]:
def extract_squat_data(results, prev_results):
    data = {"landmarks": {}, "angles": {}}

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark

        # Récupération des coordonnées des points clés
        for name, idx in KEY_LANDMARKS.items():
            data["landmarks"][name] = {
                "x": landmarks[idx].x,
                "y": landmarks[idx].y,
                "z": landmarks[idx].z
            }

        # Calcul des angles importants pour le squat
        data["angles"]["left_knee"] = calculate_angle(
            [landmarks[KEY_LANDMARKS["LEFT_HIP"]].x, landmarks[KEY_LANDMARKS["LEFT_HIP"]].y],
            [landmarks[KEY_LANDMARKS["LEFT_KNEE"]].x, landmarks[KEY_LANDMARKS["LEFT_KNEE"]].y],
            [landmarks[KEY_LANDMARKS["LEFT_ANKLE"]].x, landmarks[KEY_LANDMARKS["LEFT_ANKLE"]].y]
        )

        data["angles"]["right_knee"] = calculate_angle(
            [landmarks[KEY_LANDMARKS["RIGHT_HIP"]].x, landmarks[KEY_LANDMARKS["RIGHT_HIP"]].y],
            [landmarks[KEY_LANDMARKS["RIGHT_KNEE"]].x, landmarks[KEY_LANDMARKS["RIGHT_KNEE"]].y],
            [landmarks[KEY_LANDMARKS["RIGHT_ANKLE"]].x, landmarks[KEY_LANDMARKS["RIGHT_ANKLE"]].y]
        )

        data["angles"]["back_angle"] = calculate_angle(
            [landmarks[KEY_LANDMARKS["LEFT_SHOULDER"]].x, landmarks[KEY_LANDMARKS["LEFT_SHOULDER"]].y],
            [landmarks[KEY_LANDMARKS["LEFT_HIP"]].x, landmarks[KEY_LANDMARKS["LEFT_HIP"]].y],
            [landmarks[KEY_LANDMARKS["LEFT_KNEE"]].x, landmarks[KEY_LANDMARKS["LEFT_KNEE"]].y]
        )

    return data["angles"]

In [32]:
frames_data = []
import mediapipe as mp

# Initialisation de MediaPipe
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

prev_landmark = {}
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convertir l'image en RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Détection des points clés
    results = pose.process(frame_rgb)

    frames_data.append(data["angles"])

SyntaxError: 'return' outside function (<ipython-input-32-13165ebcb66b>, line 52)

In [25]:
import numpy as np
data1 = data
data2 = data
for el in data["landmarks"]:
  print(el)
  data["landmarks"]
  print(np.linalg.norm(np.array(data1["landmarks"][el]['x']) - np.array(data2["landmarks"][el]['x'])))

LEFT_HIP
0.0
RIGHT_HIP
0.0
LEFT_KNEE
0.0
RIGHT_KNEE
0.0
LEFT_ANKLE
0.0
RIGHT_ANKLE
0.0
LEFT_SHOULDER
0.0
RIGHT_SHOULDER
0.0


In [30]:
def has_movement(prev_points, curr_points, threshold=5):
    """
    Vérifie si le mouvement est significatif entre les deux frames en comparant la distance entre les points.
    """
    if prev_points is None:
        return True  # Premier frame, donc on considère qu'il y a du mouvement

    # for prev, curr in zip(prev_points, curr_points):
    #     if np.linalg.norm(np.array(prev) - np.array(curr)) > threshold:
    #         return True

    for el in data["landmarks"]:
      if (np.linalg.norm(np.array(prev_points[el]['x']) - np.array(curr_points[el]['x'])))> 2:
        return True
    return False

In [31]:
has_movement(data1["landmarks"], data1["landmarks"])

False